In [1]:

import gym
import gym_carla
import carla
import numpy as np 
from matplotlib import pyplot as plt
import matplotlib.image as mpimg

#!/usr/bin/env python

# Copyright (c) 2019: Jianyu Chen (jianyuchen@berkeley.edu).
#
# This work is licensed under the terms of the MIT license.
# For a copy, see <https://opensource.org/licenses/MIT>.
import random
from select import select

import tensorboard
import gym
import gym_carla
import carla
import numpy as np
from gym_carla.envs.carla_env import CarlaEnv
import pandas as pd

import torch
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from torch import nn
from collections import deque,namedtuple

import glob
import io
import base64
import os
from IPython.display import HTML
from IPython import display as ipythondisplay
from pyvirtualdisplay import Display
from gym.wrappers import Monitor

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as T

import torchvision
from torch.utils.tensorboard import SummaryWriter
from torchvision import datasets, transforms

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
class DQN(nn.Module):

    def __init__(self, state_space,action_space):
        super(DQN, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(state_space, 100),
            nn.ReLU(),
            nn.Linear(100, 50),
            nn.ReLU(),
            nn.Linear(50, 25),
            nn.ReLU(),
            nn.Linear(25,15),
            nn.ReLU(),
            nn.Linear(15, 8),
            nn.ReLU(),
            nn.Linear(8 ,action_space),
            nn.ReLU()
        )

        self.loss = nn.MSELoss()
        self.learning_rate = 0.001
        self.optimiser = optim.Adam(self.parameters(), self.learning_rate)
        self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        self.to(self.device)

    # Called with either one element to determine next action, or a batch
    # during optimization. Returns tensor([[left0exp,right0exp]...]).
    def forward(self, x):
        x = x.to(self.device)
        return self.model(x)

class ReplayMemory(object):

    def __init__(self, capacity):
        self.memory = deque(maxlen=capacity)

    def push(self, state, action, next_state, reward, done):
        # Add the tuple (state, action, next_state, reward) to the queue
        self.memory.append((state, action, next_state, reward, done))

    def sample(self, batch_size):
        batch_size = min(batch_size, len(self)) 
        return random.sample(self.memory,batch_size)

    def __len__(self):
        return len(self.memory)



def update_epsilon(epsilon):
    eps_decay = 0.99975
    epsilon = epsilon*eps_decay
    return epsilon


def select_action(net, state, epsilon):
    if epsilon > 1 or epsilon < 0:
        raise Exception('The epsilon value must be between 0 and 1')

    sample = random.random()

    with torch.no_grad():
        net.eval()
        state = torch.tensor(state, dtype=torch.float32) # Convert the state to tensor
        net_out = net(state)

    best_action = int(net_out.argmax())
    action_space_dim = net_out.shape[-1]

    if sample < epsilon:
        with torch.no_grad():
            # t.max(1) will return largest column value of each row.
            # second column on max result is index of where max element was
            # found, so we pick action with the larger expected reward.
            non_optimal_actions = [a for a in range(action_space_dim) if a != best_action]
            action = random.choice(non_optimal_actions)
            
    else:
        action = best_action
            
    net_out.cpu().numpy()

    return action


def update_step(policy_net, target_net, replay_mem, gamma, optimizer, loss_fn, batch_size):
        
    # Sample from the replay memory
    batch = replay_mem.sample(batch_size)
    batch_size = len(batch)

    # Create tensors for each element of the batch
    states      = torch.tensor([s[0] for s in batch], dtype=torch.float32, device=device)
    actions     = torch.tensor([s[1] for s in batch], dtype=torch.int64, device=device)
    rewards     = torch.tensor([s[3] for s in batch], dtype=torch.float32, device=device)

    # Compute a mask of non-final states (all the elements where the next state is not None)
    non_final_next_states = torch.tensor([s[2] for s in batch if s[2] is not None], dtype=torch.float32, device=device) # the next state can be None if the game has ended
    non_final_mask = torch.tensor([s[2] is not None for s in batch], dtype=torch.bool)

    # Compute Q values 
    policy_net.train()
    q_values = policy_net(states)
    # Select the proper Q value for the corresponding action taken Q(s_t, a)
    state_action_values = q_values.gather(1, actions.unsqueeze(1).cuda())

    # Compute the value function of the next states using the target network V(s_{t+1}) = max_a( Q_target(s_{t+1}, a)) )
    with torch.no_grad():
      target_net.eval()
      q_values_target = target_net(non_final_next_states)
    next_state_max_q_values = torch.zeros(batch_size, device=device)
    next_state_max_q_values[non_final_mask] = q_values_target.max(dim=1)[0].detach()

    # Compute the expected Q values
    expected_state_action_values = rewards + (next_state_max_q_values * gamma)
    expected_state_action_values = expected_state_action_values.unsqueeze(1)# Set the required tensor shape

    # Compute the Huber loss
    loss = loss_fn(state_action_values, expected_state_action_values)

    # Optimize the model
    optimizer.zero_grad()
    loss.backward()
    # Apply gradient clipping 
    nn.utils.clip_grad_norm_(policy_net.parameters(), 2)
    optimizer.step()

    return loss

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html
cuda:0


In [2]:
from pathlib import Path

In [3]:
from autoEncoder import *

ae = autoencoder()
ae.load_state_dict(torch.load("/home/dh26/Documents/Carla/gym-carla/AutoEncoder_new.pt"))
ae.eval()


autoencoder(
  (encoder): Sequential(
    (0): Conv2d(1, 32, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (1): LeakyReLU(negative_slope=0.02)
    (2): Conv2d(32, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (3): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (4): LeakyReLU(negative_slope=0.02)
    (5): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (6): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): LeakyReLU(negative_slope=0.02)
    (8): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (9): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): LeakyReLU(negative_slope=0.02)
    (11): Conv2d(256, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (12): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (13): LeakyReLU(negative_slope=0.02)
    (14): Conv2d(512

In [4]:
from weatherCNN import *

weather = Net()
weather.load_state_dict(torch.load("/home/dh26/Documents/Carla/gym-carla/Weather4.pt"))
weather.eval()


Net(
  (conv1): Conv2d(3, 6, kernel_size=(3, 3), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=3, stride=3, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1))
  (conv4): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=288, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=4, bias=True)
)

In [5]:
weather.eval()

Net(
  (conv1): Conv2d(3, 6, kernel_size=(3, 3), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=3, stride=3, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1))
  (conv4): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=288, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=4, bias=True)
)

In [6]:
from segmentationUNET import *
from pathlib import Path


VAL_PERCENTAGE = 0.2  # Amount of data to use for validation
TEST_NUM = 10  # Number of images to set aside for testing and visualization
NUM_CLASSES = 13  # Total number of classes in the dataset
BATCH_SIZE = 16  # Batch size for training
IMG_SIZE = 128  # The input size for model
DATA_PATH = Path('/home/dh26/Documents/Carla/gym-carla/images/')  # Location of the dataset
SHUFFLE = True  # Shuffle the dataset before making the split
LR = 0.001  # Learning rate for the model
EPOCHS = 10  # Number of epochs to train the model
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'  # Device used to train
# DEVICE = 'cpu'
unetSunny = SegmentationAgent(VAL_PERCENTAGE, TEST_NUM, NUM_CLASSES,BATCH_SIZE, IMG_SIZE, DATA_PATH, SHUFFLE, LR, DEVICE)

#unet = SegmentationAgent(val_percentage', 'test_num', 'num_classes', 'batch_size', 'img_size', 'data_path', 'shuffle_data', 'learning_rate', and 'device)
#unet = SegmentationAgent(0.2, 10,13,32,128, data_path, True, 1e-3, device)
unetSunny.model.load_state_dict(torch.load("/home/dh26/Documents/Carla/gym-carla/Models/Unet_Sunny.pt"))
unetSunny.model.eval()

SegmentationUNet(
  (conv_final): Conv2d(64, 13, kernel_size=(1, 1), stride=(1, 1))
  (down_convs): ModuleList(
    (0): DownConv(
      (conv_in): Sequential(
        (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (conv_out): Sequential(
        (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    )
    (1): DownConv(
      (conv_in): Sequential(
        (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (conv_out): Sequential(
        (0): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): Ba

In [7]:
from segmentationUNET import *
from pathlib import Path


VAL_PERCENTAGE = 0.2  # Amount of data to use for validation
TEST_NUM = 10  # Number of images to set aside for testing and visualization
NUM_CLASSES = 13  # Total number of classes in the dataset
BATCH_SIZE = 16  # Batch size for training
IMG_SIZE = 128  # The input size for model
DATA_PATH = Path('/home/dh26/Documents/Carla/gym-carla/images/')  # Location of the dataset
SHUFFLE = True  # Shuffle the dataset before making the split
LR = 0.001  # Learning rate for the model
EPOCHS = 10  # Number of epochs to train the model
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'  # Device used to train
# DEVICE = 'cpu'
unetCloudy = SegmentationAgent(VAL_PERCENTAGE, TEST_NUM, NUM_CLASSES,BATCH_SIZE, IMG_SIZE, DATA_PATH, SHUFFLE, LR, DEVICE)

#unet = SegmentationAgent(val_percentage', 'test_num', 'num_classes', 'batch_size', 'img_size', 'data_path', 'shuffle_data', 'learning_rate', and 'device)
#unet = SegmentationAgent(0.2, 10,13,32,128, data_path, True, 1e-3, device)
unetCloudy.model.load_state_dict(torch.load("/home/dh26/Documents/Carla/gym-carla/Models/Unet_Cloudy.pt"))
unetCloudy.model.eval()

SegmentationUNet(
  (conv_final): Conv2d(64, 13, kernel_size=(1, 1), stride=(1, 1))
  (down_convs): ModuleList(
    (0): DownConv(
      (conv_in): Sequential(
        (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (conv_out): Sequential(
        (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    )
    (1): DownConv(
      (conv_in): Sequential(
        (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (conv_out): Sequential(
        (0): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): Ba

In [8]:
from segmentationUNET import *
from pathlib import Path


VAL_PERCENTAGE = 0.2  # Amount of data to use for validation
TEST_NUM = 10  # Number of images to set aside for testing and visualization
NUM_CLASSES = 13  # Total number of classes in the dataset
BATCH_SIZE = 16  # Batch size for training
IMG_SIZE = 128  # The input size for model
DATA_PATH = Path('/home/dh26/Documents/Carla/gym-carla/images/')  # Location of the dataset
SHUFFLE = True  # Shuffle the dataset before making the split
LR = 0.001  # Learning rate for the model
EPOCHS = 10  # Number of epochs to train the model
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'  # Device used to train
# DEVICE = 'cpu'
unetHR = SegmentationAgent(VAL_PERCENTAGE, TEST_NUM, NUM_CLASSES,BATCH_SIZE, IMG_SIZE, DATA_PATH, SHUFFLE, LR, DEVICE)

#unet = SegmentationAgent(val_percentage', 'test_num', 'num_classes', 'batch_size', 'img_size', 'data_path', 'shuffle_data', 'learning_rate', and 'device)
#unet = SegmentationAgent(0.2, 10,13,32,128, data_path, True, 1e-3, device)
unetHR.model.load_state_dict(torch.load("/home/dh26/Documents/Carla/gym-carla/Models/Unet_HeavyRain.pt"))
unetHR.model.eval()

SegmentationUNet(
  (conv_final): Conv2d(64, 13, kernel_size=(1, 1), stride=(1, 1))
  (down_convs): ModuleList(
    (0): DownConv(
      (conv_in): Sequential(
        (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (conv_out): Sequential(
        (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    )
    (1): DownConv(
      (conv_in): Sequential(
        (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (conv_out): Sequential(
        (0): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): Ba

In [9]:
from segmentationUNET import *
from pathlib import Path


VAL_PERCENTAGE = 0.2  # Amount of data to use for validation
TEST_NUM = 10  # Number of images to set aside for testing and visualization
NUM_CLASSES = 13  # Total number of classes in the dataset
BATCH_SIZE = 16  # Batch size for training
IMG_SIZE = 128  # The input size for model
DATA_PATH = Path('/home/dh26/Documents/Carla/gym-carla/images/')  # Location of the dataset
SHUFFLE = True  # Shuffle the dataset before making the split
LR = 0.001  # Learning rate for the model
EPOCHS = 10  # Number of epochs to train the model
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'  # Device used to train
# DEVICE = 'cpu'
unetLR= SegmentationAgent(VAL_PERCENTAGE, TEST_NUM, NUM_CLASSES,BATCH_SIZE, IMG_SIZE, DATA_PATH, SHUFFLE, LR, DEVICE)

#unet = SegmentationAgent(val_percentage', 'test_num', 'num_classes', 'batch_size', 'img_size', 'data_path', 'shuffle_data', 'learning_rate', and 'device)
#unet = SegmentationAgent(0.2, 10,13,32,128, data_path, True, 1e-3, device)
unetLR.model.load_state_dict(torch.load("/home/dh26/Documents/Carla/gym-carla/Models/Unet_LightRain.pt"))
unetLR.model.eval()

SegmentationUNet(
  (conv_final): Conv2d(64, 13, kernel_size=(1, 1), stride=(1, 1))
  (down_convs): ModuleList(
    (0): DownConv(
      (conv_in): Sequential(
        (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (conv_out): Sequential(
        (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    )
    (1): DownConv(
      (conv_in): Sequential(
        (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (conv_out): Sequential(
        (0): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): Ba

In [ ]:
import torch
def main():
  # parameters for the gym_carla environment
  params = {
    'number_of_vehicles': 0,
    'number_of_walkers': 0,
    'display_size': 256,  # screen size of bird-eye render
    'max_past_step': 1,  # the number of past steps to draw
    'dt': 0.1,  # time interval between two frames
    'discrete': True,  # whether to use discrete control space
    'discrete_acc': [-3.0, 0.0, 3.0],  # discrete value of accelerations
    'discrete_steer': [-0.3, 0.0, 0.3],  # discrete value of steering angles
    'continuous_accel_range': [-3.0, 3.0],  # continuous acceleration range
    'continuous_steer_range': [-0.3, 0.3],  # continuous steering angle range
    'ego_vehicle_filter': 'vehicle.tesla.model3',  # filter for defining ego vehicle
    'port': 2000,  # connection port
    'town': 'Town03',  # which town to simulate
    'task_mode': 'curriculum',  # mode of the task, [random, roundabout (only for Town03)]
    'max_time_episode': 1000,  # maximum timesteps per episode
    'max_waypt': 12,  # maximum number of waypoints
    'obs_range': 32,  # observation range (meter)
    'lidar_bin': 0.25,  # bin size of lidar sensor (meter)
    'd_behind': 12,  # distance behind the ego vehicle (meter)
    'out_lane_thres': 2.0,  # threshold for out of lane
    'desired_speed': 3,  # desired speed (m/s)
    'max_ego_spawn_times': 200,  # maximum times to spawn ego vehicle
    'display_route': True,  # whether to render the desired route
    'pixor_size': 64,  # size of the pixor labels
    'pixor': False,  # whether to output PIXOR observation
    'dynamic_weather':True, #Set TRUE for random weather

  }

  # Set gym-carla environment
  env = gym.make('carla-v0', params=params)
  env.seed(0) # Set a random seed for the environment
  state_space_dim = 75
  action_space_dim = 3
  # Set random seeds
  torch.manual_seed(0)
  np.random.seed(0)

  gamma = 0.99   #discount factor 
  epsilon = 1
  epsilon_min = 0.01
  replay_memory_capacity = 1_000_000   
  lr = 1e-3 ##Learning rate 
  target_net_update_steps = 10  
  batch_size = 256   
  bad_state_penalty = 0   
  min_samples_for_training = 2000   

  ### Define exploration profile
##  initial_value = 5
  num_iterations = 600
#   exp_decay = np.exp(-np.log(initial_value) / num_iterations * 6) 
#   exploration_profile = [initial_value * (exp_decay ** i) for i in range(num_iterations)]

  # replay memory
  replay_mem = ReplayMemory(replay_memory_capacity)    

  # policy network
  policy_net = DQN(state_space_dim, action_space_dim).to(device)

  # target network with the same weights of the policy network
  target_net = DQN(state_space_dim, action_space_dim).to(device)
  target_net.load_state_dict(policy_net.state_dict()) # This will copy the weights of the policy network to the target network

  optimizer = torch.optim.Adam(policy_net.parameters(), lr=lr) # The optimizer will update ONLY the parameters of the policy network

  loss_fn = nn.HuberLoss()  
  #============================================LOOP============================================================================================
  
  from datetime import date
  dis_acc = params.get("discrete_acc")
  dis_steer_ang = params.get("discrete_steer")
  episode_durations=[]
  episode_scores = []
  route_scores=[]
  avg_epsilons = []
  avg_losses = []
  time_steps = 0 
  today  =date.today()
  today_date =  today.strftime("%d/%m/%Y").replace("/", "")
  writer = SummaryWriter(log_dir=f"//home//dh26//Documents//Carla//gym-carla//Models_Visual_Inputs_curriculum_{today_date}", flush_secs=30)
  weather_classes = ['Cloudy', 'Heavy Rain', 'Light Rain', 'Sunny']
  weather_conds = []
  #print(exploration_profile)
  for episode_num in range(num_iterations):  
    print(episode_num)
    w =0 
    obs = env.reset()
    action = 1
    next_state, reward, done, info = env.step(action)
    s= next_state.get("camera")
    s2 = next_state.get("camera2")
    s = torch.Tensor(s)
    s = s.reshape(1,3,128,128)
    s2 = torch.Tensor(s2)
    s2 = s2.reshape(1,3,128,128)
    i = torch.argmax(weather(s2))
    weather_cond = weather_classes[i]
    if weather_cond == 'Cloudy':
      sem = unetCloudy.model(s)
    elif weather_cond == 'Sunny':
      sem = unetSunny.model(s)
    elif weather_cond == 'Light Rain':
      sem = unetLR.model(s)
    elif weather_cond == 'Heavy Rain': 
      sem = unetHR.model(s)
    sem = torch.argmax(sem, dim=1).reshape(1,1,128,128).float()
    output,state = ae(sem)
    # print(state)
    # print(state.shape)
    #print(weather_cond)
    episode_score = 0
    episode_duration = 0
    loss_l = []
    epsilon_l = []
    state = state.cpu().detach().numpy().reshape(64)
    state = state.tolist()

    s= list(next_state.get("state"))
    s_0 = s[0]
    state.append(s_0)
    s_1 = s[1]
    state.append(s_1)
    s_2 = s[2]
    state.append(s_2)
    s_3 = s[3]
    state.append(s_3)
    # example [5.23002378e+00 -6.65688906e-01  4.13617835e-08  0.00000000e+00]
    vehicle_front = info.get("vehicle_front")
    state.append(vehicle_front)
    position = info.get("position")[0]
    state.append(position)
    angular_vel = info.get("angular_vel")
    
    angular_vel_x = angular_vel[0]
    state.append(angular_vel_x)
    angular_vel_y = angular_vel[1]
    state.append(angular_vel_y)
    angular_vel_z = angular_vel[2]
    state.append(angular_vel_z)
    acceleration = info.get("acceleration")
    state.append(acceleration)
    steer = info.get("steer")
    state.append(steer)
    #state_extra = [s_0, s_1, s_2,s_3, vehicle_front, position, angular_vel_x, angular_vel_y, angular_vel_z, acceleration, steer]
    

    while done == False:
        epsilon_l.append(epsilon)
        #print(state)
        writer.add_scalar("Epsilon_TS", epsilon, time_steps)
        action = select_action(policy_net, state, epsilon)
        if epsilon > epsilon_min:    
            epsilon = update_epsilon(epsilon)
        time_steps +=1
        next_state, reward,done,info = env.step(action)
        episode_score += reward 
        episode_duration += 1
        
        cam_info = next_state.get("camera") # example [5.23002378e+00 -6.65688906e-01  4.13617835e-08  0.00000000e+00]
        cam_info2 = next_state.get("camera2")
        s= list(next_state.get("state"))
        s_0 = s[0]

        s_1 = s[1]
        s_2 = s[2]
        s_3 = s[3]
        # example [5.23002378e+00 -6.65688906e-01  4.13617835e-08  0.00000000e+00]
        vehicle_front = info.get("vehicle_front")
        position = info.get("position")[0]
        angular_vel = info.get("angular_vel")
        angular_vel_x = angular_vel[0]
        angular_vel_y = angular_vel[1]
        angular_vel_z = angular_vel[2]
        acceleration = info.get("acceleration")
        steer = info.get("steer")
        state_extra = [s_0, s_1, s_2,s_3, vehicle_front, position, angular_vel_x, angular_vel_y, angular_vel_z, acceleration, steer]
        cam_info = torch.Tensor(cam_info)
        cam_info = cam_info.reshape(1,3,128,128)
        cam_info2 = torch.Tensor(cam_info2)
        cam_info2 = cam_info2.reshape(1,3,128,128)
        #_,weather_cond = torch.max(weather(cam_info2).data,1)
        i = torch.argmax(weather(cam_info2))
        weather_cond = weather_classes[i]
        if w == 30: 
            print(weather_cond)
            w+=1
        elif w!=30:
            w+=1
        weather_conds.append(weather_cond)
        if weather_cond == 'Cloudy':
          sem = unetCloudy.model(cam_info)
        elif weather_cond == 'Sunny':
          sem = unetSunny.model(cam_info)
        elif weather_cond == 'Light Rain':
          sem = unetLR.model(cam_info)
        elif weather_cond == 'Heavy Rain': 
          sem = unetHR.model(cam_info)
        sem = torch.argmax(sem, dim=1).reshape(1,1,128,128).float()
        output,latent_space = ae(sem)
        latent_space = latent_space.cpu().detach().numpy().reshape(64)
        next_state = latent_space.tolist()

        for i in state_extra:
          next_state.append(i)


        # Update the replay memory
        replay_mem.push(state, action, next_state, reward, done)
        
        
        if len(replay_mem) > min_samples_for_training: # we enable the training only if we have enough samples in the replay memory, otherwise the training will use the same samples too often
            loss = update_step(policy_net, target_net, replay_mem, gamma, optimizer, loss_fn, batch_size)
            writer.add_scalar("Loss",loss.item(), time_steps)
            loss_l.append(loss.item())

        if time_steps % 256 == 0:
            print('Updating target network...')
            target_net.load_state_dict(policy_net.state_dict())

        state = next_state
        #print(state)


        episode_scores.append(episode_score)
        episode_durations.append(episode_duration)

    route_score = (episode_duration/1000)*100
    print("Route Score = {}%".format(route_score))
    route_scores.append(route_score)

    if episode_num % 256 ==0:
        print('Updating target network...')
        target_net.load_state_dict(policy_net.state_dict())
    
    # if episode_num == 300  or episode_num == 600:
    #     PATH = f"//home//dh26//Documents//Carla//gym-carla//Models//model1_episode_{today_date}_{episode_num}.pth" 
    #     torch.save(policy_net.state_dict(),PATH)
    
    if len(loss_l)>0:
        avg_loss = sum(loss_l)/len(loss_l)
        avg_losses.append(avg_loss)
        writer.add_scalar('Average Loss', avg_loss, episode_num)
    elif len(loss_l) == 0:
        avg_loss = 0
        avg_losses.append(avg_loss)
        writer.add_scalar('Average Loss', avg_loss, episode_num)
    
    if len(epsilon_l)>0:
        avg_epsilon = sum(epsilon_l)/len(epsilon_l)
    elif len(epsilon_l)==0:
        avg_epsilon = 0

    avg_epsilons.append(avg_epsilon)
    # Append episode reward to a list and log stats (every given number of episodes)
    writer.add_scalar("Average_Epsilon", avg_epsilon, episode_num)
    writer.add_scalar('Episode Scores', episode_score, episode_num)
    writer.add_scalar('Episode Duration', episode_duration, episode_num)
    writer.add_scalar('Route Completion %', route_score, episode_num)
    writer.flush()
  writer.close()

  
  #d = {'Avg. Loss':avg_losses, 'Avg. Epsilon':avg_epsilons, 'Epsiode Score':episode_scores, 'Durations':episode_durations, 'Route Scores':route_scores}  
  # df = pd.DataFrame(d)
  # df.to_csv("//home//dh26//Documents//Simple_Model_Random_Metrics2.csv")
  PATH = "//home//dh26//Documents//Carla//gym-carla//Models//Model_VI_curriculum_{}.pt" .format(today_date)
  torch.save(policy_net.state_dict(),PATH)        
  env.close() 
  

if __name__ == '__main__':
  main()

connecting to Carla server...
Carla server connected!
265
0
Heavy Rain
Route Score = 5.1%
Updating target network...
1
Cloudy
Route Score = 6.2%
2
Cloudy
Route Score = 5.6000000000000005%
3
Cloudy
Route Score = 5.3%
4
Heavy Rain
Updating target network...
Route Score = 5.8999999999999995%
5
Cloudy
Route Score = 5.7%
6
Cloudy
Route Score = 5.3%
7
Cloudy
Route Score = 5.2%
8
Heavy Rain
Route Score = 5.4%
9
Updating target network...
Cloudy
Route Score = 6.6000000000000005%
10
Cloudy
Route Score = 5.8999999999999995%
11
Cloudy
Route Score = 6.1%
12
Heavy Rain
Route Score = 5.3%
13
Cloudy
Updating target network...
Route Score = 5.7%
14
Cloudy
Route Score = 5.4%
15
Cloudy
Route Score = 5.3%
16
Heavy Rain
Route Score = 6.7%
17
Cloudy
Updating target network...
Route Score = 6.9%
18
Cloudy
Route Score = 7.000000000000001%
19
Cloudy
Route Score = 10.0%
20
Light Rain
Updating target network...
Route Score = 10.5%
21
Cloudy
Route Score = 9.0%
22
Cloudy
Route Score = 13.4%
23
Updating target net

Updating target network...
Route Score = 100.0%
92
Light Rain
Updating target network...
Updating target network...
Updating target network...
Updating target network...
Route Score = 100.0%
93
Cloudy
Updating target network...
Updating target network...
Updating target network...
Updating target network...
Route Score = 100.0%
94
Cloudy
Updating target network...
Updating target network...
Updating target network...
Updating target network...
Route Score = 100.0%
95
Cloudy
Updating target network...
Updating target network...
Updating target network...
Updating target network...
Route Score = 100.0%
96
Heavy Rain
Updating target network...
Updating target network...
Updating target network...
Updating target network...
Route Score = 100.0%
97
Cloudy
Updating target network...
Updating target network...
Updating target network...
Updating target network...
Route Score = 100.0%
98
Cloudy
Updating target network...
Updating target network...
Updating target network...
Updating target net

Updating target network...
Updating target network...
Updating target network...
Updating target network...
Route Score = 100.0%
157
Cloudy
Updating target network...
Updating target network...
Updating target network...
Updating target network...
Route Score = 100.0%
158
Cloudy
Updating target network...
Updating target network...
Updating target network...
Updating target network...
Route Score = 100.0%
159
Cloudy
Updating target network...
Updating target network...
Updating target network...
Route Score = 71.8%
160
Heavy Rain
Updating target network...
Updating target network...
Updating target network...
Route Score = 100.0%
161
Updating target network...
Cloudy
Updating target network...
Updating target network...
Updating target network...
Route Score = 100.0%
162
Cloudy
Updating target network...
Updating target network...
Route Score = 34.699999999999996%
163
Cloudy
Updating target network...
Updating target network...
Updating target network...
Updating target network...
Rout

Heavy Rain
Updating target network...
Updating target network...
Updating target network...
Updating target network...
Route Score = 100.0%
224
Cloudy
Updating target network...
Updating target network...
Updating target network...
Route Score = 71.1%
225
Cloudy
Updating target network...
Updating target network...
Updating target network...
Route Score = 90.10000000000001%
226
Cloudy
Updating target network...
Updating target network...
Updating target network...
Updating target network...
Route Score = 100.0%
227
Cloudy
Updating target network...
Route Score = 15.1%
228
Cloudy
Updating target network...
Route Score = 42.8%
229
Heavy Rain
Updating target network...
Updating target network...
Updating target network...
Route Score = 65.0%
230
Cloudy
Route Score = 10.0%
231
Heavy Rain
Updating target network...
Updating target network...
Updating target network...
Updating target network...
Route Score = 100.0%
232
Heavy Rain
Updating target network...
Updating target network...
Updatin

Cloudy
Updating target network...
Updating target network...
Updating target network...
Route Score = 71.2%
293
Cloudy
Updating target network...
Updating target network...
Updating target network...
Updating target network...
Route Score = 90.4%
294
Cloudy
Updating target network...
Updating target network...
Updating target network...
Route Score = 100.0%
295
Updating target network...
Cloudy
Route Score = 13.8%
296
Cloudy
Updating target network...
Updating target network...
Updating target network...
Updating target network...
Route Score = 100.0%
297
Heavy Rain
Updating target network...
Updating target network...
Updating target network...
Updating target network...
Route Score = 96.3%
298
Cloudy
Route Score = 10.6%
299
Heavy Rain
Updating target network...
Updating target network...
Updating target network...
Updating target network...
Route Score = 100.0%
300
Heavy Rain
Updating target network...
Updating target network...
Updating target network...
Updating target network...
R